In [28]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob
import sqlite3
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

In [ ]:


for i in glob.glob("2ch/*"):
    img = Image.open(i)
    img = img.resize((100,100))
    plt.figure()
    plt.imshow(img)

In [16]:
x_train = np.empty((0,100,100,3), float)
y_train = []
conn = sqlite3.connect("naver_views.db")
cur = conn.cursor()

for i in glob.glob("2ch/*"):
    
    # 100*100にトリムして正規化
    img = Image.open(i)
    img = img.resize((100,100))
    img_array = np.array(img, 'f')
    img_array /= 255.
    
    # グレースケールは除外
    if len(img_array.shape) == 2:
        print(i);continue

    # dbからview数取得
    cur.execute("""select view from views where id = {}""".format(int(i.split('.')[0].split("/")[-1])))
    view = cur.fetchall()[0][0]

    # 追加
    x_train = np.append(x_train,img_array.reshape(-1,100,100,3),axis=0)
    y_train.append(view)
    
print(x_train.shape)
conn.close()

2ch/125.jpg
2ch/156.jpg
2ch/159.jpg
2ch/194.jpg
2ch/235.jpg
2ch/258.jpg
2ch/270.jpg
2ch/393.jpg
2ch/462.jpg
2ch/487.jpg
2ch/502.jpg
2ch/664.jpg
2ch/805.jpg
2ch/818.jpg
2ch/863.jpg
2ch/923.jpg
2ch/937.jpg
2ch/954.jpg
2ch/979.jpg
2ch/996.jpg
2ch/1116.jpg
2ch/1157.jpg
2ch/1186.jpg
2ch/1231.jpg
2ch/1272.jpg
2ch/1342.jpg
2ch/1396.jpg
2ch/1702.jpg
2ch/1794.jpg
2ch/1922.jpg
2ch/1946.jpg
2ch/1973.jpg
2ch/1974.jpg
2ch/1977.jpg
2ch/2071.jpg
2ch/2076.jpg
2ch/2245.jpg
2ch/2288.jpg
2ch/2314.jpg
2ch/2400.jpg
2ch/2434.jpg
2ch/2454.jpg
2ch/2584.jpg
2ch/2695.jpg
2ch/2832.jpg
2ch/2834.jpg
2ch/2852.jpg
2ch/2857.jpg
2ch/2920.jpg
2ch/3361.jpg
2ch/3434.jpg
2ch/3608.jpg
2ch/3672.jpg
2ch/3705.jpg
2ch/3792.jpg
2ch/3893.jpg
2ch/4016.jpg
2ch/4062.jpg
2ch/4074.jpg
2ch/4100.jpg
2ch/4103.jpg
2ch/4160.jpg
2ch/4261.jpg
2ch/4331.jpg
2ch/4366.jpg
2ch/4443.jpg
2ch/4494.jpg
2ch/4549.jpg
2ch/4605.jpg
2ch/4615.jpg
2ch/4724.jpg
2ch/4727.jpg
2ch/4811.jpg
2ch/4817.jpg
2ch/4885.jpg
2ch/4933.jpg
2ch/5090.jpg
2ch/5096.jpg
2ch/51

In [17]:
import pickle

with open("naver_dataset.dump","wb") as f:
    pickle.dump((x_train,y_train),f)

In [26]:
one_hot_y_train = []
for e,i in enumerate(y_train):
    if i > 10000:
        one_hot_y_train.append([1.,0.])
    else:
        one_hot_y_train.append([0.,1.])
    
Y_train = np.array(one_hot_y_train)[:-1000]
X_train = x_train[:-1000]
Y_test = np.array(one_hot_y_train)[-1000:]
X_test = x_train[-1000:]

print(Y_train.shape,Y_test.shape)

(11618, 2) (1000, 2)


In [29]:
nb_classes=2

# CNNを構築
model = Sequential()

# conv 3*3 -> 32 channels
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
# Max pooling 2*2
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 全結合 512
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 全結合 10
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

batch_size = 100
nb_epoch = 5
history = model.fit(X_train, Y_train,
                            batch_size=batch_size,
                            epochs=nb_epoch,
                            verbose=1,
                            validation_data=(X_test, Y_test),
                            shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 49, 49, 64)        18496     
__________